<a href="https://colab.research.google.com/github/JamesWirths/skills-introduction-to-github/blob/main/ProdSuite_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install flask flask_sqlalchemy flask_login flask_wtf pyngrok

In [2]:
import os
import logging
import time, datetime, random
from pyngrok import ngrok
from flask import Flask, jsonify, request, render_template_string, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from flask_wtf.csrf import CSRFProtect, csrf_exempt
from werkzeug.security import generate_password_hash, check_password_hash

# ------------------------------------------------------------------------------
# LOGGING CONFIGURATION
# ------------------------------------------------------------------------------
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# ------------------------------------------------------------------------------
# APP CONFIGURATION
# ------------------------------------------------------------------------------
app = Flask(__name__)
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY', 'supersecretkey')  # CHANGE THIS FOR PRODUCTION!
app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', 'sqlite:///prod_suite.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['PREFERRED_URL_SCHEME'] = 'https'
app.config['SESSION_COOKIE_SECURE'] = True

db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'login'
csrf = CSRFProtect(app)

# ------------------------------------------------------------------------------
# NGROK CONFIGURATION (Replace with your own auth token or set NGROK_AUTH_TOKEN)
# ------------------------------------------------------------------------------
ngrok_auth = os.environ.get('NGROK_AUTH_TOKEN', "2sPWW3tzDPAplI4BymN2ISGdDm1_7koYvVQgvU3He4aFnXxtm")
ngrok.set_auth_token(ngrok_auth)

# ------------------------------------------------------------------------------
# DATABASE MODELS
# ------------------------------------------------------------------------------
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password_hash = db.Column(db.String(256), nullable=False)
    xp = db.Column(db.Integer, default=0)      # Experience points
    level = db.Column(db.Integer, default=1)   # User level
    tasks = db.relationship('Task', backref='user', lazy=True)
    habits = db.relationship('Habit', backref='user', lazy=True)
    journals = db.relationship('Journal', backref='user', lazy=True)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(500))
    status = db.Column(db.String(50))  # 'todo', 'inprogress', 'done'
    completed = db.Column(db.Boolean, default=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

class Habit(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(150))
    streak = db.Column(db.Integer, default=0)
    last_done = db.Column(db.String(50))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

class Journal(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    date = db.Column(db.String(50))
    content = db.Column(db.Text)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

# ------------------------------------------------------------------------------
# HELPER FUNCTION FOR GAMIFICATION
# ------------------------------------------------------------------------------
def add_xp(user, points):
    user.xp += points
    new_level = (user.xp // 100) + 1
    if new_level != user.level:
        user.level = new_level
    db.session.commit()

# ------------------------------------------------------------------------------
# USER LOADER FOR FLASK-LOGIN
# ------------------------------------------------------------------------------
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# ------------------------------------------------------------------------------
# ROUTES: REGISTRATION, LOGIN, LOGOUT
# ------------------------------------------------------------------------------
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        if User.query.filter_by(username=username).first():
            flash("Username already exists.")
            return redirect(url_for('register'))
        new_user = User(username=username)
        new_user.set_password(password)
        db.session.add(new_user)
        db.session.commit()
        flash("Registration successful! Please log in.")
        return redirect(url_for('login'))
    return render_template_string('''
<!DOCTYPE html>
<html>
<head>
  <title>Register - ProdSuite</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
<div class="container mt-5">
  <h2>Register</h2>
  <form method="POST">
    <input type="hidden" name="csrf_token" value="{{ csrf_token() }}">
    <div class="mb-3">
      <label>Username</label>
      <input class="form-control" type="text" name="username" required>
    </div>
    <div class="mb-3">
      <label>Password</label>
      <input class="form-control" type="password" name="password" required>
    </div>
    <button class="btn btn-primary" type="submit">Register</button>
  </form>
  <p class="mt-3">Already have an account? <a href="{{ url_for('login') }}">Login here</a>.</p>
</div>
</body>
</html>
''')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method=='POST':
        username = request.form.get('username')
        password = request.form.get('password')
        user = User.query.filter_by(username=username).first()
        if user and user.check_password(password):
            login_user(user)
            flash("Logged in successfully!")
            return redirect(url_for('index'))
        else:
            flash("Invalid credentials")
            return redirect(url_for('login'))
    return render_template_string('''
<!DOCTYPE html>
<html>
<head>
  <title>Login - ProdSuite</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
<div class="container mt-5">
  <h2>Login</h2>
  <form method="POST">
    <input type="hidden" name="csrf_token" value="{{ csrf_token() }}">
    <div class="mb-3">
      <label>Username</label>
      <input class="form-control" type="text" name="username" required>
    </div>
    <div class="mb-3">
      <label>Password</label>
      <input class="form-control" type="password" name="password" required>
    </div>
    <button class="btn btn-primary" type="submit">Login</button>
  </form>
  <p class="mt-3">Don't have an account? <a href="{{ url_for('register') }}">Register here</a>.</p>
</div>
</body>
</html>
''')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash("Logged out.")
    return redirect(url_for('login'))

# ------------------------------------------------------------------------------
# API ENDPOINTS (CSRF EXEMPT for JSON calls)
# ------------------------------------------------------------------------------
@csrf.exempt
@app.route('/api/tasks', methods=['GET'])
@login_required
def api_tasks():
    user_tasks = {'todo': [], 'inprogress': [], 'done': []}
    for t in Task.query.filter_by(user_id=current_user.id).all():
        user_tasks[t.status].append({'id': t.id, 'content': t.content, 'completed': t.completed})
    return jsonify(user_tasks)

@csrf.exempt
@app.route('/api/add_task', methods=['POST'])
@login_required
def api_add_task():
    data = request.get_json()
    status = data.get('status')
    content = data.get('content')
    if status not in ['todo', 'inprogress', 'done']:
        return jsonify({"success": False, "error": "Invalid status"}), 400
    new_task = Task(content=content, status=status, user_id=current_user.id)
    db.session.add(new_task)
    db.session.commit()
    return jsonify({"success": True, "task": {"id": new_task.id, "content": new_task.content, "completed": new_task.completed}})

@csrf.exempt
@app.route('/api/edit_task', methods=['POST'])
@login_required
def api_edit_task():
    data = request.get_json()
    task_id = data.get('id')
    new_content = data.get('new_content')
    task = Task.query.filter_by(id=task_id, user_id=current_user.id).first()
    if task:
        task.content = new_content
        db.session.commit()
        return jsonify({"success": True})
    return jsonify({"success": False, "error": "Task not found"}), 404

@csrf.exempt
@app.route('/api/advance_task', methods=['POST'])
@login_required
def api_advance_task():
    data = request.get_json()
    task_id = data.get('id')
    task = Task.query.filter_by(id=task_id, user_id=current_user.id).first()
    if not task:
        return jsonify({"success": False, "error": "Task not found"}), 404
    prev_status = task.status
    if prev_status == 'todo':
        task.status = 'inprogress'
        points = 5
    elif prev_status == 'inprogress':
        task.status = 'done'
        points = 10
    else:
        return jsonify({"success": False, "error": "Task is already complete"}), 400
    db.session.commit()
    add_xp(current_user, points)
    return jsonify({"success": True, "task": {"id": task.id, "content": task.content, "status": task.status}})

@csrf.exempt
@app.route('/api/delete_task', methods=['POST'])
@login_required
def api_delete_task():
    data = request.get_json()
    task_id = data.get('id')
    task = Task.query.filter_by(id=task_id, user_id=current_user.id).first()
    if not task:
        return jsonify({"success": False, "error": "Task not found"}), 404
    db.session.delete(task)
    db.session.commit()
    return jsonify({"success": True})

@csrf.exempt
@app.route('/api/habits', methods=['GET'])
@login_required
def api_habits():
    habits_list = [{'id': h.id, 'name': h.name, 'streak': h.streak, 'last_done': h.last_done}
                   for h in Habit.query.filter_by(user_id=current_user.id).all()]
    return jsonify(habits_list)

@csrf.exempt
@app.route('/api/add_habit', methods=['POST'])
@login_required
def api_add_habit():
    data = request.get_json()
    name = data.get('name')
    new_habit = Habit(name=name, streak=0, last_done="", user_id=current_user.id)
    db.session.add(new_habit)
    db.session.commit()
    return jsonify({"success": True, "habit": {"id": new_habit.id, "name": new_habit.name, "streak": new_habit.streak, "last_done": new_habit.last_done}})

@csrf.exempt
@app.route('/api/mark_habit_done', methods=['POST'])
@login_required
def api_mark_habit_done():
    data = request.get_json()
    habit_id = data.get('id')
    today = datetime.date.today().isoformat()
    habit = Habit.query.filter_by(id=habit_id, user_id=current_user.id).first()
    if habit:
        if habit.last_done == today:
            return jsonify({"success": False, "error": "Already marked for today"}), 400
        habit.streak += 1
        habit.last_done = today
        db.session.commit()
        add_xp(current_user, 3)
        return jsonify({"success": True})
    return jsonify({"success": False, "error": "Habit not found"}), 404

@csrf.exempt
@app.route('/api/delete_habit', methods=['POST'])
@login_required
def api_delete_habit():
    data = request.get_json()
    habit_id = data.get('id')
    habit = Habit.query.filter_by(id=habit_id, user_id=current_user.id).first()
    if not habit:
        return jsonify({"success": False, "error": "Habit not found"}), 404
    db.session.delete(habit)
    db.session.commit()
    return jsonify({"success": True})

@csrf.exempt
@app.route('/api/journals', methods=['GET'])
@login_required
def api_journals():
    journals_list = [{'id': j.id, 'date': j.date, 'content': j.content}
                     for j in Journal.query.filter_by(user_id=current_user.id).all()]
    return jsonify(journals_list)

@csrf.exempt
@app.route('/api/add_journal', methods=['POST'])
@login_required
def api_add_journal():
    data = request.get_json()
    content = data.get('content')
    entry = Journal(date=datetime.date.today().isoformat(), content=content, user_id=current_user.id)
    db.session.add(entry)
    db.session.commit()
    add_xp(current_user, 2)
    return jsonify({"success": True, "journal": {"id": entry.id, "date": entry.date, "content": entry.content}})

@csrf.exempt
@app.route('/api/delete_journal', methods=['POST'])
@login_required
def api_delete_journal():
    data = request.get_json()
    journal_id = data.get('id')
    journal = Journal.query.filter_by(id=journal_id, user_id=current_user.id).first()
    if not journal:
        return jsonify({"success": False, "error": "Journal not found"}), 404
    db.session.delete(journal)
    db.session.commit()
    return jsonify({"success": True})

# ------------------------------------------------------------------------------
# MAIN PAGE ROUTE (SINGLE DEFINITION WITH GAMIFICATION INFO)
# ------------------------------------------------------------------------------
@app.route('/')
@login_required
def index():
    html = r'''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>ProdSuite – Reimagine Productivity</title>
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css2?family=Open+Sans:wght@400;600;700&display=swap" rel="stylesheet">
  <!-- Bootstrap 5 CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
  <!-- Chart.js for Dashboard -->
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <!-- Bootstrap Icons -->
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.10.3/font/bootstrap-icons.css">
  <style>
    /* Global Styles */
    body {
      font-family: 'Open Sans', sans-serif;
      margin: 0;
      padding: 0;
      background-color: #f4f7f9;
      color: #333;
    }
    a { text-decoration: none; }
    /* Navbar */
    .navbar {
      background: linear-gradient(135deg, #4facfe, #00f2fe);
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .navbar-brand, .nav-link {
      font-weight: 600;
      color: #fff !important;
    }
    .nav-link.active {
      border-bottom: 2px solid #fff;
    }
    /* Gamification Info in Navbar */
    .gamification-info {
      margin-left: auto;
      color: #fff;
      font-weight: bold;
    }
    /* Hero Section */
    .hero {
      background: url('https://source.unsplash.com/1920x1080/?technology,workspace') no-repeat center center/cover;
      height: 100vh;
      position: relative;
      display: flex;
      align-items: center;
      justify-content: center;
    }
    .hero::before {
      content: "";
      position: absolute;
      top: 0; left: 0;
      width: 100%; height: 100%;
      background-color: rgba(0, 0, 0, 0.5);
    }
    .hero-content {
      position: relative;
      z-index: 2;
      text-align: center;
      color: #fff;
      padding: 0 15px;
    }
    .hero-content h1 {
      font-size: 4rem;
      margin-bottom: 20px;
    }
    .hero-content p {
      font-size: 1.5rem;
      margin-bottom: 30px;
    }
    .btn-cta {
      font-size: 1.2rem;
      padding: 15px 30px;
    }
    /* Features Section */
    .features {
      padding: 60px 0;
      background-color: #fff;
    }
    .features .feature {
      text-align: center;
      padding: 20px;
    }
    .features .feature i {
      font-size: 3rem;
      color: #00aaff;
      margin-bottom: 15px;
    }
    .features .feature h3 {
      font-size: 1.75rem;
      margin-bottom: 10px;
    }
    .features .feature p {
      font-size: 1rem;
    }
    /* App Sections */
    .section {
      padding: 60px 0;
    }
    .card {
      border: none;
      border-radius: 10px;
      box-shadow: 0 4px 8px rgba(0,0,0,0.08);
      margin-bottom: 20px;
      background: #fff;
    }
    /* Color-Coded Task Styles */
    .task-todo {
      background-color: #fff3cd;
      border: 1px solid #ffeeba;
      color: #856404;
    }
    .task-inprogress {
      background-color: #f8d7da;
      border: 1px solid #f5c6cb;
      color: #721c24;
    }
    .task-done {
      background-color: #d4edda;
      border: 1px solid #c3e6cb;
      color: #155724;
    }
    /* Pomodoro Overlay */
    #pomodoro-overlay {
      position: fixed;
      top: 0;
      left: 0;
      width: 100%; height: 100%;
      background-color: rgba(0,0,0,0.9);
      color: #fff;
      display: none;
      align-items: center;
      justify-content: center;
      flex-direction: column;
      z-index: 10000;
      transition: opacity 0.3s ease;
    }
    #pomodoro-overlay h1 {
      font-size: 6rem;
      margin-bottom: 20px;
    }
    /* Footer */
    .footer {
      background-color: #333;
      color: #fff;
      padding: 30px 0;
      text-align: center;
    }
    .back-to-top {
      cursor: pointer;
      color: #00f2fe;
      font-weight: bold;
    }
  </style>
</head>
<body>
  <!-- Navigation Bar -->
  <nav class="navbar navbar-expand-lg">
    <div class="container">
      <a class="navbar-brand" data-section="tasks" href="#">ProdSuite</a>
      <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarContent">
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbarContent">
        <ul class="navbar-nav me-auto">
          <li class="nav-item"><a class="nav-link" data-section="tasks" href="#">Tasks</a></li>
          <li class="nav-item"><a class="nav-link" data-section="habits" href="#">Habits</a></li>
          <li class="nav-item"><a class="nav-link" data-section="journal" href="#">Journal</a></li>
          <li class="nav-item"><a class="nav-link" data-section="mindfulness" href="#">Mindfulness</a></li>
          <li class="nav-item"><a class="nav-link" data-section="dashboard" href="#">Dashboard</a></li>
          <li class="nav-item"><a class="nav-link" href="/logout">Logout</a></li>
        </ul>
        <span class="navbar-text gamification-info">
          Level: {{ current_user.level }} | XP: {{ current_user.xp }} / {{ current_user.level * 100 }}
          <div class="progress" style="width: 100px; height: 10px; display:inline-block; vertical-align:middle;">
            <div class="progress-bar" role="progressbar" style="width: {{ (current_user.xp % 100) }}%;" aria-valuenow="{{ (current_user.xp % 100) }}" aria-valuemin="0" aria-valuemax="100"></div>
          </div>
        </span>
      </div>
    </div>
  </nav>

  <!-- Hero Section -->
  <section class="hero" id="hero">
    <div class="hero-content">
      <h1>Reimagine Your Productivity</h1>
      <p>Manage tasks, track habits, journal your progress, and stay focused—all in one sleek suite.</p>
      <button id="get-started" class="btn btn-primary btn-cta">Get Started</button>
    </div>
  </section>

  <!-- Features Section -->
  <section class="features">
    <div class="container">
      <div class="row">
        <div class="col-md-4 feature">
          <i class="bi bi-check-circle"></i>
          <h3>Organize Tasks</h3>
          <p>Color-coded tasks with action icons make management effortless.</p>
        </div>
        <div class="col-md-4 feature">
          <i class="bi bi-lightbulb"></i>
          <h3>Boost Focus</h3>
          <p>Integrated Pomodoro sessions and mindfulness features keep you on track.</p>
        </div>
        <div class="col-md-4 feature">
          <i class="bi bi-graph-up"></i>
          <h3>Track Progress</h3>
          <p>Visual dashboards provide insight into your productivity metrics.</p>
        </div>
      </div>
    </div>
  </section>

  <!-- Main App Sections -->
  <div class="container my-4">
    <!-- Tasks Section -->
    <div class="section" id="tasks">
      <div class="card p-3">
        <div class="card-header">Task Manager</div>
        <div class="row">
          <div class="col-md-4">
            <h6>To Do</h6>
            <div id="todo-list" class="list-group"></div>
            <button class="btn btn-outline-primary mt-2 w-100" onclick="addTask('todo')">+ Add Task</button>
          </div>
          <div class="col-md-4">
            <h6>In Progress</h6>
            <div id="inprogress-list" class="list-group"></div>
            <button class="btn btn-outline-primary mt-2 w-100" onclick="addTask('inprogress')">+ Add Task</button>
          </div>
          <div class="col-md-4">
            <h6>Done</h6>
            <div id="done-list" class="list-group"></div>
            <button class="btn btn-outline-primary mt-2 w-100" onclick="addTask('done')">+ Add Task</button>
          </div>
        </div>
        <hr>
        <button class="btn btn-warning" id="start-pomodoro">Start Pomodoro</button>
      </div>
    </div>

    <!-- Habits Section -->
    <div class="section" id="habits">
      <div class="card p-3">
        <div class="card-header">Habit Tracker</div>
        <div id="habit-list" class="list-group"></div>
        <button class="btn btn-outline-primary mt-2" onclick="addHabit()">+ Add Habit</button>
      </div>
    </div>

    <!-- Journal Section -->
    <div class="section" id="journal">
      <div class="card p-3">
        <div class="card-header">Journal</div>
        <div id="journal-list" class="list-group"></div>
        <textarea id="journal-entry" class="form-control my-2" placeholder="Write your journal entry here..."></textarea>
        <button class="btn btn-outline-primary" onclick="addJournal()">Add Entry</button>
      </div>
    </div>

    <!-- Mindfulness Section -->
    <div class="section" id="mindfulness">
      <div class="card p-3">
        <div class="card-header">Mindfulness</div>
        <p class="mt-2">Take a moment to breathe and reflect. Here's an inspirational quote:</p>
        <blockquote id="quote" class="blockquote"></blockquote>
        <button class="btn btn-secondary" onclick="newQuote()">New Quote</button>
      </div>
    </div>

    <!-- Dashboard Section -->
    <div class="section" id="dashboard">
      <div class="card p-3">
        <div class="card-header">Productivity Dashboard</div>
        <canvas id="dashboardChart"></canvas>
      </div>
    </div>
  </div>

  <!-- Pomodoro Overlay -->
  <div id="pomodoro-overlay">
    <h1 id="pomodoro-timer">25:00</h1>
    <button class="btn btn-danger btn-lg mt-3" id="cancel-pomodoro">Cancel Pomodoro</button>
  </div>

  <!-- Footer with Back-to-Top -->
  <footer class="footer">
    <div class="container">
      <p>&copy; 2025 ProdSuite. All Rights Reserved.</p>
      <p class="back-to-top" onclick="scrollToTop()">Back to Top</p>
    </div>
  </footer>

  <!-- Bootstrap JS Bundle -->
  <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
  <script>
    // Back-to-Top functionality
    function scrollToTop() {
      window.scrollTo({ top: 0, behavior: 'smooth' });
    }

    // "Get Started" button: scroll to Tasks and hide the hero.
    document.getElementById('get-started').addEventListener('click', () => {
      document.getElementById('tasks').scrollIntoView({ behavior: 'smooth' });
      document.getElementById('hero').style.display = 'none';
    });

    // Navigation: Each nav link scrolls directly to its section.
    document.querySelectorAll('.nav-link').forEach(link => {
      link.addEventListener('click', () => {
        let section = link.getAttribute('data-section');
        document.getElementById(section).scrollIntoView({ behavior: 'smooth' });
      });
    });

    // ----------------- TASKS -----------------
    function loadTasks() {
      fetch('/api/tasks')
        .then(res => res.json())
        .then(data => {
          ['todo', 'inprogress', 'done'].forEach(status => {
            let listEl = document.getElementById(status + '-list');
            listEl.innerHTML = '';
            data[status].forEach(task => {
              let row = document.createElement('div');
              row.className = 'list-group-item d-flex justify-content-between align-items-center';
              if (status === 'todo') row.classList.add('task-todo');
              else if (status === 'inprogress') row.classList.add('task-inprogress');
              else if (status === 'done') row.classList.add('task-done');

              let span = document.createElement('span');
              span.innerText = task.content;
              row.appendChild(span);

              let iconsDiv = document.createElement('div');

              let checkIcon = document.createElement('i');
              checkIcon.className = 'bi bi-check-circle-fill me-2';
              checkIcon.style.cursor = 'pointer';
              checkIcon.title = 'Advance Task';
              checkIcon.addEventListener('click', () => {
                advanceTask(task.id, status);
              });
              iconsDiv.appendChild(checkIcon);

              let trashIcon = document.createElement('i');
              trashIcon.className = 'bi bi-trash-fill';
              trashIcon.style.cursor = 'pointer';
              trashIcon.title = 'Delete Task';
              trashIcon.addEventListener('click', () => {
                deleteTask(task.id);
              });
              iconsDiv.appendChild(trashIcon);

              row.appendChild(iconsDiv);
              listEl.appendChild(row);
            });
          });
        });
    }

    function addTask(status) {
      let content = prompt("Enter task:");
      if(content) {
        fetch('/api/add_task', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({status: status, content: content})
        }).then(() => loadTasks());
      }
    }

    function advanceTask(taskId, currentStatus) {
      fetch('/api/advance_task', {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify({id: taskId})
      }).then(() => loadTasks());
    }

    function deleteTask(taskId) {
      if (confirm("Are you sure you want to delete this task?")) {
        fetch('/api/delete_task', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({id: taskId})
        }).then(() => loadTasks());
      }
    }

    // ----------------- HABITS -----------------
    function loadHabits() {
      fetch('/api/habits')
        .then(res => res.json())
        .then(data => {
          let list = document.getElementById('habit-list');
          list.innerHTML = '';
          data.forEach(habit => {
            let item = document.createElement('div');
            item.className = 'list-group-item d-flex justify-content-between align-items-center';
            let habitText = document.createElement('span');
            habitText.innerText = habit.name + " (Streak: " + habit.streak + ")";
            item.appendChild(habitText);

            let btnDiv = document.createElement('div');
            let doneBtn = document.createElement('button');
            doneBtn.className = 'btn btn-success btn-sm me-2';
            doneBtn.innerText = "Done Today";
            doneBtn.addEventListener('click', () => markHabitDone(habit.id));
            btnDiv.appendChild(doneBtn);

            let trashIcon = document.createElement('i');
            trashIcon.className = 'bi bi-trash-fill';
            trashIcon.style.cursor = 'pointer';
            trashIcon.title = 'Delete Habit';
            trashIcon.addEventListener('click', () => deleteHabit(habit.id));
            btnDiv.appendChild(trashIcon);

            item.appendChild(btnDiv);
            list.appendChild(item);
          });
        });
    }

    function addHabit() {
      let name = prompt("Enter habit name:");
      if(name) {
        fetch('/api/add_habit', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({name: name})
        }).then(() => loadHabits());
      }
    }

    function markHabitDone(id) {
      fetch('/api/mark_habit_done', {
        method: 'POST',
        headers: {'Content-Type': 'application/json'},
        body: JSON.stringify({id: id})
      }).then(() => loadHabits());
    }

    function deleteHabit(id) {
      if (confirm("Are you sure you want to delete this habit?")) {
        fetch('/api/delete_habit', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({id: id})
        }).then(() => loadHabits());
      }
    }

    // ----------------- JOURNAL -----------------
    function loadJournals() {
      fetch('/api/journals')
        .then(res => res.json())
        .then(data => {
          let list = document.getElementById('journal-list');
          list.innerHTML = '';
          data.forEach(entry => {
            let item = document.createElement('div');
            item.className = 'list-group-item d-flex justify-content-between align-items-center';
            let entryText = document.createElement('span');
            entryText.innerHTML = "<strong>" + entry.date + ":</strong> " + entry.content;
            item.appendChild(entryText);

            let trashIcon = document.createElement('i');
            trashIcon.className = 'bi bi-trash-fill';
            trashIcon.style.cursor = 'pointer';
            trashIcon.title = 'Delete Journal Entry';
            trashIcon.addEventListener('click', () => deleteJournal(entry.id));
            item.appendChild(trashIcon);

            list.appendChild(item);
          });
        });
    }

    function addJournal() {
      let entry = document.getElementById('journal-entry').value;
      if(entry) {
        fetch('/api/add_journal', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({content: entry})
        }).then(() => {
          document.getElementById('journal-entry').value = '';
          loadJournals();
        });
      }
    }

    function deleteJournal(id) {
      if (confirm("Are you sure you want to delete this journal entry?")) {
        fetch('/api/delete_journal', {
          method: 'POST',
          headers: {'Content-Type': 'application/json'},
          body: JSON.stringify({id: id})
        }).then(() => loadJournals());
      }
    }

    // ----------------- MINDFULNESS / POMODORO -----------------
    let pomodoroTimer;
    let pomodoroTimeLeft = 25 * 60; // in seconds
    function startPomodoro() {
      document.getElementById('pomodoro-overlay').style.display = 'flex';
      document.getElementById('start-pomodoro').disabled = true;
      pomodoroTimer = setInterval(() => {
        if(pomodoroTimeLeft <= 0) {
          clearInterval(pomodoroTimer);
          alert("Pomodoro complete!");
          resetPomodoro();
        } else {
          pomodoroTimeLeft--;
          updatePomodoroDisplay();
        }
      }, 1000);
    }

    function updatePomodoroDisplay() {
      let minutes = Math.floor(pomodoroTimeLeft / 60);
      let seconds = pomodoroTimeLeft % 60;
      document.getElementById('pomodoro-timer').innerText =
        String(minutes).padStart(2, '0') + ":" + String(seconds).padStart(2, '0');
    }

    function resetPomodoro() {
      clearInterval(pomodoroTimer);
      pomodoroTimeLeft = 25 * 60;
      updatePomodoroDisplay();
      document.getElementById('pomodoro-overlay').style.display = 'none';
      document.getElementById('start-pomodoro').disabled = false;
    }

    // ----------------- DASHBOARD -----------------
    function loadDashboard() {
      let ctx = document.getElementById('dashboardChart').getContext('2d');
      new Chart(ctx, {
        type: 'bar',
        data: {
          labels: ['Tasks Done', 'Habit Streak (avg)', 'Journal Entries'],
          datasets: [{
            label: 'Productivity Metrics',
            data: [
              Math.floor(Math.random() * 10),
              Math.floor(Math.random() * 10),
              Math.floor(Math.random() * 10)
            ],
            backgroundColor: ['rgba(75,192,192,0.6)', 'rgba(153,102,255,0.6)', 'rgba(255,159,64,0.6)']
          }]
        },
        options: { scales: { y: { beginAtZero: true } } }
      });
    }

    // ----------------- MINDFULNESS (QUOTES) -----------------
    function newQuote() {
      const quotes = [
        "Believe you can and you're halfway there.",
        "Stay positive, work hard, make it happen.",
        "Don't watch the clock; do what it does. Keep going."
      ];
      document.getElementById('quote').innerText = quotes[Math.floor(Math.random() * quotes.length)];
    }

    // ----------------- INITIALIZE THE APP -----------------
    function init() {
      loadTasks();
      loadHabits();
      loadJournals();
      loadDashboard();
      newQuote();
      document.getElementById('start-pomodoro').addEventListener('click', startPomodoro);
      document.getElementById('cancel-pomodoro').addEventListener('click', resetPomodoro);
    }
    init();
  </script>
</body>
</html>
    '''
    return render_template_string(html, current_user=current_user)

# ------------------------------------------------------------------------------
# TESTING COMMAND
# ------------------------------------------------------------------------------
@app.cli.command('test')
def test():
    """Run tests."""
    import unittest
    class ProdSuiteTestCase(unittest.TestCase):
        def setUp(self):
            app.config['TESTING'] = True
            app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
            self.app = app.test_client()
            with app.app_context():
                db.create_all()
                test_user = User(username="testuser")
                test_user.set_password("testpass")
                db.session.add(test_user)
                db.session.commit()
                self.test_user = test_user

        def login(self, username, password):
            return self.app.post('/login', data=dict(username=username, password=password), follow_redirects=True)

        def test_registration_and_login(self):
            rv = self.app.post('/register', data=dict(username="newuser", password="newpass"), follow_redirects=True)
            self.assertIn(b"Registration successful", rv.data)
            rv = self.login("newuser", "newpass")
            self.assertIn(b"Logged in successfully", rv.data)

        def test_advance_and_delete_task(self):
            self.login("testuser", "testpass")
            rv = self.app.post('/api/add_task', json={'status': 'todo', 'content': 'Test Task'}, follow_redirects=True)
            data = rv.get_json()
            task_id = data['task']['id']
            rv = self.app.post('/api/advance_task', json={'id': task_id}, follow_redirects=True)
            data = rv.get_json()
            self.assertEqual(data['task']['status'], 'inprogress')
            rv = self.app.post('/api/delete_task', json={'id': task_id}, follow_redirects=True)
            data = rv.get_json()
            self.assertTrue(data['success'])
    suite = unittest.TestLoader().loadTestsFromTestCase(ProdSuiteTestCase)
    unittest.TextTestRunner(verbosity=2).run(suite)

# ------------------------------------------------------------------------------
# MAIN: RUN THE APP VIA NGROK
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    with app.app_context():
        # For development purposes, drop all tables so that the schema is updated.
        db.drop_all()   # WARNING: This deletes all data!
        db.create_all()
    public_url = ngrok.connect(5000)
    print("ngrok tunnel available at:", public_url)
    app.run()


ImportError: cannot import name 'csrf_exempt' from 'flask_wtf.csrf' (/usr/local/lib/python3.11/dist-packages/flask_wtf/csrf.py)